<h1><center>
Toxic Comments Kaggle Competition Data Preperation
</center></h1>
<h2><center>
Preparing the Data to Use in Modeling
</center></h2>
<h3><center>
Marcel Colvin 912033961
</center></h3>

In [1]:
import pandas as pd
import numpy as np
from gensim.models import word2vec
import nltk
from nltk import corpus
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer as wnl
import re
import gensim
from sklearn.neural_network import MLPClassifier
import time
from ast import literal_eval

In [2]:
#nltk.download()

In [3]:
train = pd.read_csv('Toxic_comments/train.csv')
test = pd.read_csv('Toxic_comments/test.csv')

In [4]:
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

### Investigate the data

In [5]:
print('Total comments: {}'.format(len(train)))
print('toxic: {}'.format(train[train['toxic'] > 0]['toxic'].count()))
print('severe_toxic: {}'.format(train[train['severe_toxic'] > 0]['severe_toxic'].count()))
print('obscene: {}'.format(train[train['obscene'] > 0]['obscene'].count()))
print('threat: {}'.format(train[train['threat'] > 0]['threat'].count()))
print('insult: {}'.format(train[train['insult'] > 0]['insult'].count()))
print('identity_hate: {}'.format(train[train['identity_hate'] > 0]['identity_hate'].count()))

Total comments: 159571
toxic: 15294
severe_toxic: 1595
obscene: 8449
threat: 478
insult: 7877
identity_hate: 1405


In [6]:
train['length'] = train['comment_text'].str.len()

In [2]:
def lemmatize(word):
    lemmatized = wnl().lemmatize(word.lower())
    return word
def tokenize(text):
    new_text = re.sub("[^0-9a-zA-Z]", ' ', text)
    new_list = new_text.lower().split()
    stop = stopwords.words('english')
    words = [x for x in new_list if not x in stop]
    final = [lemmatize(x) for x in words]
    return final
train['tokens'] = train["comment_text"].apply(tokenize)
test['tokens'] = test['comment_text'].apply(tokenize)

In [3]:
google_model = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

In [4]:
def get_average_vector(tokens, vector, k=300):
    if len(tokens)<1:
        return np.zeros(k)
    vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens]
    average_vec = np.divide(np.sum(vectorized, axis=0), len(vectorized))
    return average_vec

#train["avg_word"] = list(train['tokens'].apply(lambda x: get_average_vector(x, google_model)))
#test["avg_word"] = list(test['tokens'].apply(lambda x: get_average_vector(x, google_model)))

In [10]:
train.to_csv("training_data.csv", index = False)
test.to_csv("test_data.csv", index = False)

In [ ]:
import glob
import pandas as pd
path = '/home/marcel/Github/Toxic_comments/nyt-comments/comments/'
for file_name in glob.glob(path+'*.csv'):
    print(file_name)
    nyt_comments = pd.read_csv(file_name)
    nyt_comments['tokens'] = nyt_comments['commentBody'].apply(tokenize)
    nyt_comments["avg_word"] = list(nyt_comments['tokens'].apply(lambda x: get_average_vector(x, google_model)))
    nyt_comments.to_csv(file_name.split('.')[0] + '_updated.'+ file_name.split('.')[1])

/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsMarch2018.csv


/home/marcel/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (14,15,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsApril2018.csv


/home/marcel/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
